# 1.导入数据

In [31]:
# 导入相关库
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [32]:
# data = pd.read_csv('srbct.csv').iloc[:,1:]
# data['group'] = data['y'].map({'EWS':1,'RMS':2,'NB':3,'BL':4})
data = pd.read_csv('glass.csv')

In [33]:
data

,RI,Na,Mg,AI,Si,K,Ca,Ba,Fe,group
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [4]:
# data = data.rename(columns={9:'group'})

In [34]:
X = np.array(data.iloc[:,:-1])
y = np.array(data['group'])

In [35]:
gamma = []
for i in range(X.shape[0]-1):
    for j in range(i+1,X.shape[0]):
               gamma.append(np.linalg.norm(X[i] - X[j])**2)
                   
gamma = 1 / (np.median(gamma) / 2)

# 2.定义评估指标CER和目标函数obj

In [36]:
def cer(c1:list,c2:list):
    if len(c1) != len(c2):
        print('length error')
        return None
    else:
        n = len(c1)
        res = 0
        for i in range(1,n):
            for j in range(i):
                res += abs(int(c1[i]==c1[j]) - int(c2[i]==c2[j]))
        res /= n*(n-1)/2
        return res

In [37]:
def skkm_obj(X,c,w,gamma):
    n = X.shape[0]
    p = X.shape[1]
    c_list = list(set(c))
    res = 0
    for k in c_list:
        k_index = (c == k)
        X_k = X[k_index]*w
        n_k = X_k.shape[0]

        expanded_matrix = np.tile(X_k, (n_k, 1))
        
        # 计算差异
        diff_matrix = expanded_matrix - np.repeat(X_k, n_k, axis=0)

        result = diff_matrix.reshape(-1, p)
        res += np.exp((-1)*gamma*(np.linalg.norm(result,axis=1)**2)).sum()
    res -= n
    res /= n
    return res

In [38]:
with open('seed_list.txt','r') as f:
    a = f.readlines()
    seed_list = [int(x.replace('\n','')) for x in a]
seed_list

[1640, 7588, 1595, 8103, 5010, 7342, 2839, 8523, 2544, 3876]

In [39]:
result_df = pd.DataFrame({'seed':seed_list})
result_df['指标向量'] = None
result_df['CER'] = None
result_df = result_df.set_index('seed')
result_df

,指标向量,CER
seed,,
1640,None,None
7588,None,None
1595,None,None
8103,None,None
5010,None,None
7342,None,None
2839,None,None
8523,None,None
2544,None,None


# 3.原SKKM算法

In [ ]:
# 定义参数
n_features = 7 # 选择特征数
n_clusters = 6 # 聚类数
tol = 0.0001 # 收敛阈值

# # 生成模拟数据
# # X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_clusters, random_state=1)
noise = np.random.normal(0, 0.1, size=(214, 100))
X = np.hstack((X, noise))
n = X.shape[0]
p = X.shape[1]

# 初始化聚类结果
for seed in seed_list:
    # 初始化指标向量为全0向量
    w_new = np.zeros(p)
    np.random.seed(seed)
    y_pred = np.random.choice(list(range(1,n_clusters)),size=n)
    i = 0
    while True:
        # 固定分区求最优指标向量
        w_old = w_new.copy()
        w_new = np.zeros(p)
        while w_new.sum() < n_features:
            # 初始化最大的目标函数值为负无穷
            max_obj = -float('inf')

            for j in range(p):
                if w_new[j] == 1:
                    continue

                # 更新指标向量
                w_new[j] = 1

                # 更新目标函数
                new_obj = skkm_obj(X,y_pred,w_new,gamma)
                # 计算聚类结果和质心
                # kmeans.fit(new_K)
                # new_labels = kmeans.labels_ 
                # new_centroids = kmeans.cluster_centers_ 
                # print(kmeans.inertia_)
                # print(kmeans.transform(X.T))

                #计算目标函数值
                # center = kmeans.cluster_centers_.mean(axis=0)
                # new_obj = np.linalg.norm(new_K-center)**2/n -kmeans.inertia_ / n

                # 如果新的目标函数值大于当前最大的目标函数值，更新目标函数值和特征索引
                if new_obj >= max_obj:
                    max_obj = new_obj
                    best_j = j

                # 还原指标向量
                w_new[j] = 0
            w_new[best_j] = 1

        kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=300, tol=0.0001, algorithm='full')
        X_w = X*w_new
        K = rbf_kernel(X_w, gamma=gamma)
        kmeans.fit(K)
        y_pred = kmeans.labels_ 
        i += 1
        if np.sum(np.abs(w_new - w_old)) / np.sum(np.abs(w_old)) < tol:
            break
    # 输出结果
    result_df.loc[seed,'指标向量'] = w_new
    result_df.loc[seed,'CER'] = cer(y,y_pred)


In [ ]:
result_df.to_excel('result.xlsx')

# 4.基于遗传算法改进

## 遗传算法定义

In [26]:
pc = 0.02                          # pc为变异的概率
t = 20  #遗传算法迭代的次数
m = 100 # 种群的个体数,要求大于20以保证具有随机性

#遗传算法
def GA(d):
    population = np.zeros((m,p))     # 初始化种群
    for i in range(m):                # 定义种群的个体数为 m
        a = np.zeros(p-d)
        b = np.ones(d)                # 将选择的d维特征定义为个体c中的1
        c = np.append(a,b)
        c = (np.random.permutation(c.T)).T    # 随机生成一个d维的个体
        population[i] = c             # 初代的种群为 population，共有m个个体
        
    #遗传算法的迭代次数为t
    fitness_change = np.zeros(t)
    for i in range(t):
        fitness = np.zeros(m)             # fitness为每一个个体的适应度值
        for j in range(m):
            fitness[j] = skkm_obj(X,y_pred,population[j],gamma)          # 计算每一个体的适应度值   
        population = selection(population,fitness)  # 通过概率选择产生新一代的种群
        population = crossover(population)          # 通过交叉产生新的个体
        population = mutation(population)           # 通过变异产生新个体
        fitness_change[i] = max(fitness)      #找出每一代的适应度最大的染色体的适应度值
        # print(f'第{i}轮迭代结束')
    # 随着迭代的进行，每个个体的适应度值应该会不断增加，所以总的适应度值fitness求平均应该会变大
    print(fitness_change)
    best_fitness = max(fitness)
    best_people = population[fitness.argmax()]
    
    return best_people,best_fitness,fitness_change,population
    
    


#轮盘赌选择
def selection(population,fitness):
    fitness_sum = np.zeros(m)
    for i in range(m):
        if i==0:
            fitness_sum[i] = fitness[i]
        else:
            fitness_sum[i] = fitness[i] + fitness_sum[i-1]
    for i in range(m):
        fitness_sum[i] = fitness_sum[i] / sum(fitness)
    
    #选择新的种群
    population_new = np.zeros((m,p))
    for i in range(m):
        rand = np.random.uniform(0,1)
        for j in range(m):
            if j==0:
                if rand<=fitness_sum[j]:
                    population_new[i] = population[j]
            else:
                if fitness_sum[j-1]<rand and rand<=fitness_sum[j]:
                    population_new[i] = population[j]
    return population_new
                

#交叉操作
def crossover(population):
    father = population[0:m//2,:]
    mother = population[m//2:,:]
    np.random.shuffle(father)       # 将父代个体按行打乱以随机配对
    np.random.shuffle(mother)
    for i in range(m//2):
        father_1 = father[i]
        mother_1 = mother[i]
        one_zero = []
        zero_one = []
        for j in range(p):
            if father_1[j]==1 and mother_1[j]==0:
                one_zero.append(j)
            if father_1[j]==0 and mother_1[j]==1:
                zero_one.append(j)
        length1 = len(one_zero)
        length2 = len(zero_one)
        length = max(length1,length2)
        half_length = int(length/2)        #half_length为交叉的位数 
        for k in range(half_length):       #进行交叉操作
            z = one_zero[k]
            q = zero_one[k]
            father_1[z]=0
            mother_1[z]=1
            father_1[q]=1
            mother_1[q]=0
        father[i] = father_1               #将交叉后的个体替换原来的个体
        mother[i] = mother_1
    population = np.append(father,mother,axis=0)
    return population
                
            
    
#变异操作
def mutation(population):
    for i in range(m):
        c = np.random.uniform(0,1)
        if c<=pc:
            mutation_s = population[i]
            zero = []                           # zero存的是变异个体中第几个数为0
            one = []                            # one存的是变异个体中第几个数为1
            for j in range(p):
                if mutation_s[j]==0:
                    zero.append(j)
                else:
                    one.append(j)
                    
            if (len(zero)!=0) and (len(one)!=0):
                a = np.random.randint(0,len(zero))    # e是随机选择由0变为1的位置
                b = np.random.randint(0,len(one))     # f是随机选择由1变为0的位置
                e = zero[a]
                f = one[b]
                mutation_s[e] = 1
                mutation_s[f] = 0
                population[i] = mutation_s
            
    return population

## 训练

In [18]:
result_df_ga = pd.DataFrame({'seed':seed_list})
result_df_ga['指标向量'] = None
result_df_ga['CER'] = None
result_df_ga = result_df_ga.set_index('seed')

In [27]:
n_clusters =6 # 聚类数
d = 7   # d为要选择的特征个数
tol = 0.0001 # 收敛阈值
# 生成模拟数据
# X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_clusters, random_state=1)
noise = np.random.normal(0, 0.1, size=(214, 100))
X = np.hstack((X, noise))
n = X.shape[0]
p = X.shape[1]
for seed in seed_list:
    # 初始化指标向量为全0向量
    w_new = np.zeros(p)
    # 初始化聚类结果
    np.random.seed(seed)
    y_pred = np.random.choice(list(range(1,n_clusters)),size=n)
    i = 0
    w_new = np.zeros(p)
    while True:
        w_old = w_new.copy()
        best_people,best_fitness,fitness_change,best_population = GA(d) 
        w_new = best_people

        kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=300, tol=0.0001, algorithm='full')
        X_w = X[:,w_new==1]
        K = rbf_kernel(X_w, gamma=gamma)
        kmeans.fit(K)
        y_pred = kmeans.labels_ 
        i += 1
        print(f'GA-SKMM第{i}轮迭代结束')

        if np.sum(np.abs(w_new - w_old)) / np.sum(np.abs(w_old)) < tol:
            break
        if i==50:
            break
    # 输出结果
    result_df_ga.loc[seed,'指标向量'] = w_new
    result_df_ga.loc[seed,'CER'] = cer(y,y_pred)

[41.03272496 41.11135891 41.06938511 41.05571243 41.13032291 40.88669255
 40.88047932 40.8716646  40.88702706 40.89777593 40.90054152 40.86888233
 40.89916295 40.90586057 40.86582987 40.88283673 40.86224403 40.90302663
 40.88038208 40.87035414]
GA-SKMM第1轮迭代结束
[53.26520014 53.2552137  53.2774804  53.24258247 53.23413269 53.20865189
 53.27051887 53.2522482  53.28393345 53.27728672 53.29444864 53.24851398
 53.29142465 53.28957245 53.28944315 53.28534899 53.30040484 53.27586227
 53.29519175 53.30118884]
GA-SKMM第2轮迭代结束
[41.50361765 41.59442866 41.59442866 41.5513106  41.61273137 41.61026165
 41.4276744  41.4379538  41.6037894  41.35792396 41.47863857 41.43761749
 41.45544015 41.53951871 41.41486823 41.44339414 41.46090374 41.47999004
 41.42771658 41.43773137]
GA-SKMM第3轮迭代结束
[42.30473909 42.27113852 42.24730319 42.29769749 42.20355258 42.29375444
 42.14153337 42.19341672 42.15675891 42.24193287 42.20247599 42.22966854
 42.21422348 42.21235396 42.2303406  42.17655654 42.13670115 42.25364208
 

In [ ]:
# best_d = np.zeros(p-1)          # judge存的是每一个维数的最优适应度


# for d in range(7,8):            # d为从60个特征中选择的特征维数
#     best_people,best_fitness,fitness_change,best_population = GA(d)     # fitness_change是遗传算法在迭代过程中适应度变化
#     best_d[d-1] = best_fitness     # best是每一维数迭代到最后的最优的适应度，用于比较
#     print("在取%d维的时候，通过遗传算法得出的最优适应度值为：%.6f"%(d,best_fitness))


# '''
# 若要看遗传算法的收敛情况，则看在d=30的情况下的fitness_change就可以
# '''


# '''
# #画图
# x = np.arange(0,59,1)
# plt.xlabel('dimension')
# plt.ylabel('fitness')
# plt.ylim((0,0.3))            # y坐标的范围
# plt.plot(x,best_d,'r')
# # plt.savefig("Sonar_best_d.jpg",dpi=2000)
# '''

# x = np.arange(0,t,1)
# plt.xlabel('dimension')
# plt.ylabel('fitness')
# # plt.ylim((0,0.1))            # y坐标的范围
# plt.plot(x,fitness_change,'r')


In [28]:
result_df_ga.to_excel('result_ga.xlsx')

In [29]:
result_df_ga['CER'].mean()

0.3618928524417533

In [ ]:
result_df['CER'].mean()

In [30]:
result_df_ga

,指标向量,CER
seed,,
1640,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.38796
7588,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0.355667
1595,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.355798
8103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.36317
5010,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0.366724
7342,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.359265
2839,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.36102
8523,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.352288
2544,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.366811
